In [1]:
import sys
sys.path.append("./expectation_value")

from expectationvalue import ExpVal
import numpy as np
import qiskit.quantum_info as qif
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

In [2]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver


class MoleculeHamiltonian():
    def __init__(self,
                 molecule,
                 converter,
                 basis,
                ):
        self.molecule = molecule
        self.basis = basis
        driver = PySCFDriver(
            atom= self.molecule,
            basis=self.basis,
            charge=0,
            spin=0,
            unit=DistanceUnit.ANGSTROM,
            )
        
        self.problem = driver.run()
        self.converter = converter
        
    def Hamiltonian(self,
                    freeze_core = True,
                    remove_orbitals = None
                   ):
        problem = self.problem
        fermionic_op = self.problem.hamiltonian.second_q_op()
            
        if freeze_core:
            fc_transformer = FreezeCoreTransformer(freeze_core=freeze_core, remove_orbitals=remove_orbitals)
            problem = fc_transformer.transform(problem)
            fermionic_op = problem.hamiltonian.second_q_op()
            
        qubit_op = self.converter.convert(fermionic_op,
                                           sector_locator=problem.symmetry_sector_locator)
        return qubit_op
    
    def ComputeGroundState(self , circuit = True):
        solver = NumPyMinimumEigensolver()
        
        ground_solver = GroundStateEigensolver(self.converter, solver)
        ground_state = ground_solver.solve(self.problem)
        
        state = ground_state.groundstate[0]
        eigenvalue = ground_state.groundenergy
        
        if circuit:
            return state, eigenvalue
        else:
            return qif.Statevector(state).data, eigenvalue

In [8]:
def get_number_nbody_terms(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    bodies : list.
        N-bodies interactions.
    """
    
    n_bodies_inter = []
    
    for i in range(len(hamiltonian)):
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        num_x = pauli_string.count('X')
        num_y = pauli_string.count('Y')
        
        n_body_terms = num_x+num_y
        
        n_bodies_inter.append(n_body_terms)
    
    bodies = list(set(n_bodies_inter))
    
    return bodies


def get_obs_data(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    coeffs : array(num_paulis).
        Coefficients of each pauli string of the hamiltonian.
    obs : array(2, 2, n_qubits, num_paulis)
        Observable made of every pauli string of the hamiltonian.
    """
    
    num_paulis = len(hamiltonian)
    n_qubits = hamiltonian.num_qubits

    paulis_dict = {'X': np.array([[0., 1.], [1., 0.]], dtype="complex"),
               'Y': np.array([[0., -1.*1j], [1.*1j, 0.]], dtype="complex"),
               'Z': np.array([[1., 0], [0, -1.]], dtype="complex"),
               'I': np.array([[1., 0], [0, 1.]], dtype="complex")}

    obs = np.zeros((2, 2, n_qubits, num_paulis), dtype=complex)
    coeffs = np.zeros(num_paulis, dtype=complex)

    for i in range(num_paulis):
        pauli_string_coeff = hamiltonian[i].coeffs[0]
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        pauli_string_list = [paulis_dict[i] for i in pauli_string]
        obs[:,:,:,i] =  np.stack(pauli_string_list, axis=-1)
        coeffs[i] = pauli_string_coeff
    
    return coeffs, obs

In [9]:
#########################################################################################################

In [10]:
dist = 1.0
molecules = {
    "H2Be": "Be .0 .0 .0; H .0 .0 -" + str(dist) + "; H .0 .0 " + str(dist),
    "H2":"H .0 .0 .0; H .0 .0 " + str(dist),
    "LiH":"Li .0 .0 .0; H .0 .0 " + str(dist),
    "H2O": "H -0.0399 -0.0038 0.0; O 1.5780 0.8540 0.0; H 2.7909 -0.5159 0.0",
    "NH4": "N 0.0 0.0 0.149; H 0.0 0.947 -0.348; H  0.821 -0.474 -0.348; H -0.821 -0.474 -0.348"
}

In [11]:
# mol = 'H2'
# mol = 'H2Be'
mol = 'LiH'
# mol = "H2O"
# mol = "NH4"

In [12]:
converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = None )
problem = molecular_hamiltonian.problem
n_qubits = hamiltonian.num_qubits
bodies = get_number_nbody_terms(hamiltonian)

In [13]:
type(hamiltonian)

qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp

In [14]:
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()
eig_gs

-9.371991912791243

In [ ]:
results = []

for i in range(100):
    algorithm = ExpVal(n_shots = 300000,
            bodies = bodies, 
            r = 9, 
            r_shots = 1000,
            n_qubits = n_qubits)

    algorithm.get_interferences(circ_gs)

    coeffs, obs = get_obs_data(hamiltonian)

    exp_val = algorithm.exp_val(obs)

    print('Exp val calculado: ', np.sum(exp_val*coeffs).real)
    results.append(np.sum(exp_val*coeffs).real)

Exp val calculado:  -9.369668300217091
Exp val calculado:  -9.368752411889309
Exp val calculado:  -9.366922049876614
Exp val calculado:  -9.372038872947638
Exp val calculado:  -9.368319668561474
Exp val calculado:  -9.364757602092215
Exp val calculado:  -9.369064257571454
Exp val calculado:  -9.369815507669623
Exp val calculado:  -9.362833760632517
Exp val calculado:  -9.369594206678439
Exp val calculado:  -9.366495794020828
Exp val calculado:  -9.369848021378823
Exp val calculado:  -9.36741644995471
Exp val calculado:  -9.362213839695475
Exp val calculado:  -9.36562120794275
Exp val calculado:  -9.367145351793516
Exp val calculado:  -9.366473947145167
Exp val calculado:  -9.37269838977472
Exp val calculado:  -9.367769125034718
Exp val calculado:  -9.3634459646961
Exp val calculado:  -9.364018281886029
Exp val calculado:  -9.370499397837484
Exp val calculado:  -9.365769558657183
Exp val calculado:  -9.372134313966155
Exp val calculado:  -9.36423928390248
Exp val calculado:  -9.36842617

In [ ]:
np.abs(results-eig_gs)

In [ ]:
np.mean(np.abs(results-eig_gs))